# Aleasim Simple Control Test

In [ ]:
%load_ext autoreload
%autoreload 2

from alea.sim.epa.disturbance_model import DisturbanceModel
from alea.sim.math_lib import Quaternion

import logging

import alea.sim
import numpy as np
import scipy as sp
import skyfield

from datetime import datetime
import time
from alea.sim.epa.earth_magnetic_field import EarthMagneticFieldModel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel
from alea.sim.kernel.kernel import AleasimKernel
from alea.sim.spacecraft.spacecraft import Spacecraft
import matplotlib.pyplot as plt
import logging
import cProfile
import pstats
from pstats import SortKey
from matplotlib import pyplot as plt
import skyfield.sgp4lib

from typing import Dict

import numpy as np


from alea.sim.kernel.frames import ReferenceFrame, FrameTransformation
from alea.sim.spacecraft.actuators.simple_actuators import SimpleActuator, SimpleMagnetorquer
from alea.sim.spacecraft.sensors import SimpleMagSensor, SimpleSunSensor, SimpleGyroSensor

from alea.sim.spacecraft.eps.solar_panel import SolarPanelModel
from alea.sim.spacecraft.eps.power_system import PowerSystemModel
from alea.sim.spacecraft.eps.eps import EPSModel

logging.basicConfig(level=logging.INFO)

In [ ]:
%load_ext autoreload
%autoreload 2

sim_dt=1e-2 #s
control_dt=0.1 #s
duration=200.0 #s

control_sample_rate = int(1/control_dt)
kernel = AleasimKernel(dt=sim_dt, date=2024.2)

adyn = AttitudeDynamicsModel(kernel)
odyn = OrbitDynamicsModel(kernel)
magm = EarthMagneticFieldModel(kernel)
DModel = DisturbanceModel(kernel)

sc = Spacecraft(kernel, ctrl_sample_period=control_dt)

rwx, rwy, rwz = sc._rws
mtqx, mtqy, mtqz =  sc._mtqs

mag_sens = sc._mag_sens
sun_sens = sc._sun_sens
gyro = sc._gyro_sens

power_sys = PowerSystemModel(kernel)

solar_panels: list[SolarPanelModel] = power_sys.solar_panels
eps = power_sys.eps
eps._force_state_of_charge(50)

kernel.add_model(sc)
kernel.add_model(magm)
kernel.add_model(odyn)
kernel.add_model(adyn)
kernel.add_model(DModel)
kernel.add_model(power_sys)

adyn.set_state(np.array([0.1,0.756,0.1,0,0,0,0.0]))

kernel.advance(duration)

kernel.kill()

## AOCS Control Summary Plots

In [ ]:
print(f'mean error {np.mean(sc.state_array[:,14])}')
objs = plt.plot(sc.time_array, sc.state_array[:,8:11])
plt.legend(iter(objs), sc.saved_state_element_names[8:11])
plt.title("Absolute Knowledge Error")
plt.xlabel('Time (s)')
plt.ylabel('degrees')
plt.show()

objs = plt.plot(sc.time_array, sc.state_array[:,11:14])
plt.legend(iter(objs), sc.saved_state_element_names[11:14])
plt.title("Absolute Pointing Error")
plt.xlabel('Time (s)')
plt.ylabel('degrees')
plt.show()

fig, axs = plt.subplots(4)
for i in range(4):
    ax = axs[i]
    ax.plot(sc.time_array, sc.state_array[:,i], label=f'q{i}_target', linestyle='dashed')
    ax.plot(sc.time_array, sc.state_array[:,4+i], label=f'q{i}_estimated', linestyle='dashed')
    ax.plot(adyn.time_array, adyn.state_array[:,i], label=f'q{i}_true',linewidth=3)
    ax.legend()
plt.suptitle("Estimated and Real Quaternion Elements")
plt.xlabel('Time (s)')
plt.show()

objs = plt.plot(adyn.time_array, adyn.state_array[:,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.ylabel('rad/s')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(adyn.time_array, adyn.state_array[:,7:10])
plt.title("Angular Acceleration")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[7:10]))
plt.ylabel('rad/s^2')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(adyn.time_array, adyn.state_array[:,10:])
plt.title("Disturbance Torques")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[10:13]))
plt.ylabel('Nm')
plt.xlabel('Time (s)')
plt.show()


for rw, id in zip(sc._rws, ['x', 'y', 'z']):
    plt.plot(rw.time_array, rw.state_array[:,0], label=id)
plt.title("Reaction Wheel Torques")
plt.legend()
plt.ylabel('Nm')
plt.xlabel('Time (s)')
plt.show()


for mtq, id in zip(sc._mtqs, ['x', 'y', 'z']):
    plt.plot(mtq.time_array, mtq.state_array[:,0], label=id)
plt.title("Mtq Dipole Moments")
plt.legend()
plt.ylabel('Am^2')
plt.xlabel('Time (s)')
plt.show()

sensors = [gyro, sun_sens, mag_sens]
for sens in sensors:
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    for i in range(3):
        ax1.plot(sens.time_array, sens.state_array[:,i], label=sens.saved_state_element_names[i])
        ax2.plot(sens.time_array, sens.state_array[:,i+sens.axes], label=sens.saved_state_element_names[i+sens.axes])
        ax1.legend()
        ax2.legend()
    plt.suptitle(f'{sens.name} Measurements vs. Ground Truth')
    plt.xlabel('Time (s)')
    plt.show()

# Reaction Wheel Plots

In [ ]:
##['torque_rw', 'torque_cmd_rw', 'torque_cmd_no_delay_rw', 'acceleration_rw', 'friction_torque_rw', 'velocity_rw', 'momentum_rw', 'current_rw', 'power_rw']
for rw in sc._rws:
    axs = plt.plot(rw.time_array,rw.state_array[:, 0], label=rw.saved_state_element_names[0]+ rw.name)
plt.title("Wheel Torque")
plt.ylabel('Torque [Nm]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in sc._rws:
    axs = plt.plot(rw.time_array,rw.state_array[:, 1], label=rw.saved_state_element_names[1]+ rw.name)
plt.title("Wheel Torque Command")
plt.ylabel('Torque [Nm]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()


for rw in sc._rws:
    plt.plot(rw.time_array,rw.state_array[:,3], label=rw.saved_state_element_names[3]+ rw.name)
plt.title("Wheel Acceleration")
plt.ylabel('Accel [rad/s^2]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in sc._rws:
    plt.plot(rw.time_array,rw.state_array[:,4], label=rw.saved_state_element_names[4]+ rw.name)
plt.title("Friction Torque")
plt.ylabel('Torque [Nm]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in sc._rws:
    plt.plot(rw.time_array,rw.state_array[:,5], label=rw.saved_state_element_names[5]+ rw.name)
plt.title("Wheel Velocity")
plt.ylabel('Velocity [rad/s]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in sc._rws:
    plt.plot(rw.time_array,rw.state_array[:,6], label=rw.saved_state_element_names[6]+ rw.name)
plt.title("Wheel Momentum")
plt.ylabel('Momentum [kg m^2 rad/s]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for rw in sc._rws:
    plt.plot(rw.time_array,rw.state_array[:,7], label=rw.saved_state_element_names[7] + rw.name)
plt.title("Wheel Current Draw")
plt.ylabel('Current [A]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for mtq in sc._mtqs:
    plt.plot(mtq.time_array, mtq.state_array[:,0], label=mtq.saved_state_element_names[0] + mtq.name)
plt.title("Magnetorquer Moments")
plt.ylabel('Moment [Am^2]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

for mtq in sc._mtqs:
    plt.plot(mtq.time_array, mtq.state_array[:,4], label=mtq.saved_state_element_names[4] + mtq.name)
plt.title("Magnetorquer Current Draw")
plt.ylabel('Moment [Am^2]')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

## Orbit Plots

In [ ]:
print(f'data elements: {odyn.saved_state_element_names}')
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(odyn.state_array[:,0],odyn.state_array[:,1], odyn.state_array[:,2])
ax.set_xlabel('X ECI [m]')
ax.set_ylabel('Y ECI [m]')
ax.set_zlabel('Z ECI [m]')
plt.show()

plt.title("ECI Position")
objs = plt.plot(odyn.time_array, odyn.state_array[:,0:3])
plt.legend(iter(objs), tuple(odyn.saved_state_element_names[0:3]))
plt.xlabel('Time (s)')
plt.show()

plt.title("ECI Velocity")
objs = plt.plot(odyn.time_array, odyn.state_array[:,3:6])
plt.legend(iter(objs), tuple(odyn.saved_state_element_names[3:6]))
plt.xlabel('Time (s)')
plt.show()

plt.title("Lon, Lat")
objs = plt.plot(odyn.time_array, odyn.state_array[:,6:8])
plt.legend(iter(objs), tuple(odyn.saved_state_element_names[6:8]))
plt.xlabel('Time (s)')
plt.show()

plt.title("WGS84 Altitude")
plt.plot(odyn.time_array, odyn.state_array[:,8])
plt.xlabel('Time (s)')
plt.show()

## Energy Plots

In [ ]:
#solar panel states - ['pwr_gen','energy_gen_total']
for panel in solar_panels:
    plt.plot(panel.time_array, panel.state_array[:,0], label=panel.name)
plt.legend()
plt.title('Power Generated per Panel')
plt.ylabel('Power (W)')
plt.xlabel('Time (s)')
plt.show()

#eps states: ['battery_soc_percent', 'battery_soc_Ah', 'watt_hour_estimate', 'solar_current', 'load_current', 'eps_voltage', 'power_net', 'power_in', 'power_out']
plt.plot(eps.time_array, eps.state_array[:,0])
plt.title('Eps State of Charge %')
plt.ylabel('%')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,1])
plt.title('Eps State of Charge Amp Hours')
plt.ylabel('Ah')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,2])
plt.title('Eps Watt Hours (inst.)')
plt.ylabel('Ah')
plt.xlabel('Time (s)')
plt.show()

objs = plt.plot(eps.time_array, eps.state_array[:,3:5])
plt.legend(iter(objs), tuple(eps.saved_state_element_names[3:5]))
plt.title('Currents')
plt.ylabel('A')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,5])
plt.title('Eps voltage')
plt.ylabel('V')
plt.xlabel('Time (s)')
plt.show()

for i in range(6,9):
    plt.plot(eps.time_array, eps.state_array[:, i], label=eps.saved_state_element_names[i])
mean_power_in = eps.state_array[:, 7].mean()
mean_power_out = eps.state_array[:, 8].mean()
plt.hlines(mean_power_in, eps.time_array[0], eps.time_array[-1], label='Average Power In', colors='g', linestyles='--')
plt.hlines(mean_power_out, eps.time_array[0], eps.time_array[-1], label='Average Power Out', colors='r', linestyles='--')
plt.title("EPS Power")
plt.legend()
plt.ylabel('W')
plt.xlabel('Time (s)')
plt.show()